# **Case Study**: How Schmidt Technology can identify high risk supplier in their supply chain using their ERP data
<br><br><br>


**Situation**:<br>
    The German Supply Chain Due Diligence Act requires large German companies to monitor, assess, and address human rights and environmental risks in their supply chains, ensuring compliance from direct suppliers and taking action if risks appear further down the chain.

**Complication**:<br>
Schmidt Medical technology does currently not have a methodology to assess the risks in their supply chains.

**Question**:<br>
How can Schmidt assess the risk of their suppliers using their ERP data?

**Answer**:<br>
The answer is through a quantitative risk score for each of their suppliers, which allows Schmidt to assess which suppliers the need to ask for certificates and potential audit, conduct remedial measures with or potentially offboard.


This jupyter notebook covers the following:
- [1. What is the result of the risk calculation?](#result-of-the-risk-calculation)
- [2. What was the approach I took?](#the-approach-i-took)
- [3. What are the next steps?](#the-next-steps)
<br><br><br>

---


# **The Approach I Took**

1. [Organize Data By Source](#organize-data-by-source)
2. [ETL: Extract, Transform, Load](#etl)
3. [Create Join Table](#create-country-mapping)
4. [Query Data For Analysis](#create-country-mapping)
5. [Engineer Metrics](#transform-data-into-partial-scores)

### **Organize Data By Source**

The provided data can be organized into two main categories, as shown in the table below:

| **Category**              | **Description**                                                                                                                                                                                                                                                                                              |
|---------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **A) Customer-Specific Data** | This schema includes tables with information unique to Schmidt, which may vary for other customers:<br> - `addresses.csv`<br> - `articles.csv`<br> - `orders.csv`<br><br>The customer-specific data does not directly contain risk-related variables. |
| **B) External Data**      | This category includes tables with risk-related data applicable to all suppliers, regardless of whether a specific Tacto customer (e.g., Schmidt) currently engages with them:<br> - `indices.csv`<br> - `suppliers.csv`<br><br>Risk-related fields include:<br> - `indices.human_rights_index`<br> - `indices.environmental_risk`<br> - `suppliers.certificates_valid` |
| **C) Join Table**         | This category includes tables used to join or merge `Customer-Specific Data` with `External Data`, based on shared keys or indices.  <br><br> In our case, this join table will be `country_mapping.csv`. However, since `country_mapping.csv` is not provided, it will need to be created manually or sourced from an appropriate external dataset




### **ETL**

- [1. Extract](#extract)
- [2. Transform](#transform)
- [3. Load](#load)

### **Extract**
We are given .csv files, so we extract them using 

```python
pd.read_csv(path)
```

### **Load**
We are loading our transformed data into our mysql database using 

```python
df.to_sql(table_name, engine, if_exists='replace', index=False) for every table.
```

That's why we need to create the database a priori

In [1]:
import sys
sys.path.append("..")
from src.db_utils import reset_database, ENGINE as engine

reset_database(database_name="tacto")

### **Transform**
- [Objective](#objective)
- [Reusable Cleaning Steps](#reusable-cleaning-steps)
- [Handling Missing Data](#handling-missing-data)


#### Objective
To maintain consistency, the database must accurately represent the structure and relationships within the underlying data. This includes labeling identical entities uniformly and enforcing database rules, such as uniqueness, to avoid duplication and inconsistencies. By establishing consistency and standardized naming conventions, the database becomes more reliable and easier to manage.

Data cleaning tasks should be applied early in the data pipeline during the initial loading phase to ensure the data is manageable from the start. Below is an outline of the key data cleaning steps.

#### Reusable Cleaning Steps
Some typical data cleaning operations can be applied across multiple tables. We can streamline these processes by defining reusable functions and applying them to each CSV file. Key tasks include:

1. **Removing duplicates** based on ID columns.
2. **Standardizing capitalization** in object (string) columns for uniformity.
3. **Trimming extra spaces and quotes** around string values to avoid formatting inconsistencies.
4. **Converting numerical strings** to integers or floats where appropriate.

#### Handling Missing Data
When dealing with missing values, different approaches should be considered for customer-specific data versus external risk-related data.

- **Customer-Specific Data**: We can automate standard missing-value handling (e.g., mean imputation or dropping missing values) because the risk of introducing bias is lower.
  
- **External Data**: For tables like `indices.csv` that contain country-specific risk metrics (e.g., `human_rights_index` and `environmental_risk`), automated imputation can introduce bias. Here, missing values may fall under specific missingness patterns:
  - **MAR (Missing At Random)**: Missing values could be related to other observed values, meaning certain values are missing in specific countries.
  - **MNAR (Missing Not At Random)**: The absence of values might be directly related to the values themselves. For example, countries with low human rights or environmental risk scores might be less likely to report these metrics.

To address missing values without introducing bias, we propose two strategies:

- **A) Conservative Imputation**: Replace missing values with the minimum existing value, ensuring we don’t overestimate scores in the absence of data.
- **B) Correlation-Based Imputation**: Since `human_rights_index` and `environmental_risk` appear correlated, we can estimate missing values by creating a regression model between these two variables. This approach would use the observed relationship to predict missing values, minimizing the introduction of bias.

For the sake of simplicity we are going to pick **Conservative Imputation**.

In [2]:
from src.data_cleaning import process_dataframe, fill_missing_with_min
import pandas as pd
import os

file_paths = [
    'data/raw/customer/addresses.csv',
    'data/raw/customer/articles.csv',
    'data/raw/customer/orders.csv',

    'data/raw/external/indices.csv',
    "data/raw/external/suppliers.csv"
]


for file_path in file_paths:
    df = pd.read_csv(os.path.join("..", file_path))
    df = process_dataframe(df)

    if os.path.basename(file_path) == 'indices.csv':
        # Replace missing values in 'human_rights_index' and 'enivronmental_risk' with the minimum existing value
        for col in ['human_rights_index', 'enivronmental_risk']:
            df = fill_missing_with_min(df, ['human_rights_index', 'enivronmental_risk'])


    table_name = os.path.basename(file_path).split('.')[0]
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    
    print(f'{table_name} table created successfully')

addresses table created successfully
articles table created successfully
orders table created successfully
indices table created successfully
suppliers table created successfully


### **Create Country Mapping**

#### MVP: Unscalable Solution
For Schmidt Medical Technology, manually mapping the `country` field from their internal supplier data file `addresses.csv` to the `country_id` column in `indices.csv` (which includes risk scores) is manageable because Schmidt’s supplier portfolio includes only a limited number of unique countries.

#### Scaling for 100+ Customers
As we onboard more customers, a scalable solution is necessary. We can automate this mapping using a publicly available dataset, such as [IBAN’s country codes dataset](https://www.iban.com/country-codes).

However, variations in naming conventions within customer data—such as “USA,” “U.S.,” or “United States” (and potential typos)—may prevent direct matching, as mapping tables require exact spelling matches.

To handle these discrepancies, we can add an intermediate step of fuzzy matching customer data country names to those in the dataset. 

This can be achieved with `fuzzywuzzy.process` for basic fuzzy matching or, for improved accuracy, by applying `cosine_similarity` on embeddings generated by large language models (LLMs). For example, using `sentence_transformers.SentenceTransformer("paraphrase-MiniLM-L6-v2")` or **OpenAI API** embeddings.




In [3]:
from sqlalchemy import Column, String
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()


# Define ORM model
class CountryMapping(Base):
    __tablename__ = 'country_mapping'
    country_id = Column(String(3), primary_key=True)  # Set length for VARCHAR
    country_name = Column(String(50))


# Create the table in the database (drop first if exists)
Base.metadata.drop_all(engine, tables=[CountryMapping.__table__], checkfirst=True)
Base.metadata.create_all(engine)

# Define mapping data and insert in a single transaction
mapping_data = [
    {'country_id': 'CHN', 'country_name': 'China'},
    {'country_id': 'DEU', 'country_name': 'Germany'},
    {'country_id': 'SWE', 'country_name': 'Sweden'},
    {'country_id': 'ISR', 'country_name': 'Israel'},
    {'country_id': 'CHE', 'country_name': 'Switzerland'}
]

session = sessionmaker(bind=engine)()
session.bulk_insert_mappings(CountryMapping, mapping_data)  
session.commit()
session.close()

### Transform Data Into Partial Scores

In [4]:
from src.db_utils import fetch_query_as_dataframe

df = fetch_query_as_dataframe("../sql/query_risk_model_variables.sql")
df

,supplier_id,sum_of_order_volume,human_rights_index,environmental_risk,total_company_revenue,certificates_valid,status
0,1080,147236.0,39.0,27.0,7.008456e+07,Yes,Active
1,1088,182387.0,39.0,27.0,2.571650e+07,Yes,Active
2,1072,345386.0,39.0,27.0,1.682030e+08,No,Active
3,1050,396517.0,39.0,27.0,1.637613e+08,Yes,Active
4,1097,414846.0,39.0,27.0,1.721610e+08,No,Active
...,...,...,...,...,...,...,...
70,1082,608010.0,98.0,91.0,2.079393e+08,Yes,Active
71,1063,883136.0,98.0,91.0,4.186066e+08,Yes,Active
72,1052,260070.0,98.0,91.0,5.695522e+08,No,Active
73,1069,559736.0,98.0,91.0,1.329933e+09,Yes,Active


In [5]:
from src.metrics_engineering import engineer_metrics

metrics_df = engineer_metrics(df)
metrics_df

,supplier_id,social_score,environmental_score,financial_score,regulatory_score,operational_score,independance_score
0,1080,39,27,52,100,100,91
1,1088,39,27,41,100,100,89
2,1072,39,27,62,0,100,77
3,1050,39,27,62,100,100,73
4,1097,39,27,63,0,100,72
...,...,...,...,...,...,...,...
70,1082,98,91,65,100,100,58
71,1063,98,91,73,100,100,38
72,1052,98,91,76,0,100,83
73,1069,98,91,86,100,100,62


# **Result Of The Risk Calculation**
- [Step 1: Select Importance Weights](#step-1-select-importance-weights)<br>
- [Step 2: Choose the risk calculation strategy](#step-2-choose-the-risk-calculation-strategy)<br>
- [Step 3: Calculate the risk score & show results](#step-3-calculate-the-risk-score--show-results)<br>

### **Step 1: Select Importance Weights**

There is no single, universally ideal quantitative measure of risk. Rather, the most appropriate measure of risk for each company is one that is tailored to its specific needs and philosophy. This customization includes two key choices:

1. [Select Risk Component Weights](#select-risk-component-weights)
2. [Choose the Risk Component Formula](#choose-the-risk-calculation-strategy)

The risk component weights can be further detailed based on these types of risk:

- **Financial Risk**: The possibility that a supplier’s financial instability could disrupt the supply chain, requiring companies to monitor for signs of potential financial failure to ensure supply continuity.

- **Regulatory & Legal Risk**: The risk that suppliers may violate laws and regulations, potentially causing the company to face liability or penalties under LkSG for failing to enforce compliance.

- **Operational Risk**: The chance of a supplier experiencing process failures or logistical issues, which requires oversight to avoid bottlenecks that could hinder legal compliance.

- **Environmental Risk**: Under LkSG, companies must assess and address supplier practices related to environmental harm, such as high CO₂ emissions or unsustainable resource use, with non-compliance risking regulatory action by the German state.

- **Social Risk**: Social risks, including human rights violations or unsafe labor practices, are critical under LkSG. Companies must enforce standards to prevent issues like child labor, forced labor, and unsafe working conditions, or they could face significant legal consequences.

- **Dependancy Risk**: Relying heavily on a single supplier for the majority of transaction volume creates a singular point of failure. If that supplier faces disruptions—whether due to financial instability, operational issues, or regulatory non-compliance—it could severely impact the company’s entire supply chain, leaving limited alternatives to maintain production or service levels and increasing vulnerability to supply interruptions.

In [6]:
from IPython.display import display
from src.visualization import create_risk_sliders


risk_categories = [col for col in metrics_df.columns if not col in ["supplier_id", "dependance"]]

slider_widget, slider_dict = create_risk_sliders(risk_categories)
display(slider_widget)

### **Step 2: Choose the risk calculation strategy**

- **Weighted Sum**:
In the Weighted Sum Strategy, each risk factor (e.g., financial risk, operational risk) is assigned a weight representing its importance. The overall risk score is calculated by multiplying each factor by its weight and then summing the results.

    - Assumes a linear relationship between risk factors
    - underestimating critical issues
    - intuitive
<br>
- **Weighted Geometric Mean**:
The Weighted Geometric Mean Strategy calculates the overall risk score by first normalizing each risk factor to a [0, 1] scale, raising each factor to the power of its weight, and then multiplying them together. 

    - If any risk factor is zero, the total score becomes zero
    - captures the compounded effect of multiple risk factors
    - less intuitive

In [7]:
from src.risk_scoring import get_strategy_selector_widget

get_strategy_selector_widget()

Dropdown(description='Select Strategy:', options={'Weighted Sum': <src.risk_scoring.WeightedSumStrategy object…

Selected strategy: WeightedGeometricMeanStrategy


### **Step 3: Calculate the risk score & show results**

In [9]:
from src.risk_scoring import calculate_quantitative_risk_score, WeightedSumStrategy, WeightedGeometricMeanStrategy

strategy = WeightedSumStrategy()
weights = {k: v.value/100 for k, v in slider_dict.items()}

risk_score = calculate_quantitative_risk_score(metrics_df, **weights)

risk_score = pd.concat([metrics_df["supplier_id"], risk_score], axis=1)
risk_score.sort_values(by="quantitative_risk_score", ascending=False, inplace=True)

output_path = '../data/output/risk_score.csv'
risk_score.to_csv(output_path, index=False)

import pandas as pd
from src.visualization import style_dataframe

supplier_df = fetch_query_as_dataframe("../sql/query_supplier_data.sql")
supplier_df["sum_of_order_volume"] = supplier_df["sum_of_order_volume"].astype(int)

risk_df = pd.read_csv("../data/output/risk_score.csv")

total_df = pd.merge(supplier_df, risk_df, left_on="supplier_id", right_on="supplier_id")
total_df = pd.merge(total_df, metrics_df, left_on="supplier_id", right_on="supplier_id")
total_df.sort_values(by="quantitative_risk_score", ascending=False, inplace=True)
style_dataframe(total_df)

,supplier_id,sum_of_order_volume,country,Industry,quantitative_risk_score,social_score,environmental_score,financial_score,regulatory_score,operational_score,independance_score
29,1032,397423,Sweden,Metals,92,98,91,89,100,100,73
38,1034,393419,Germany,Metals,91,87,96,90,100,100,74
40,1029,161727,Germany,Metals,91,87,96,74,100,100,90
31,1010,265402,Sweden,Metals,90,98,91,69,100,100,83
2,1093,357168,Germany,Plastics,90,87,96,79,100,100,76
74,1036,141831,Germany,Electronics,90,87,96,68,100,100,92
24,1075,624376,Switzerland,Metals,89,98,91,96,100,100,57
25,1069,559736,Switzerland,Metals,89,98,91,86,100,100,62
36,1068,472902,Germany,Metals,89,87,96,87,100,100,68
35,1012,536728,Germany,Metals,89,87,96,92,100,100,63


In [10]:
from src.visualization import plot_choropleth_subplots
indices_df = fetch_query_as_dataframe("../sql/query_indices.sql")
plot_choropleth_subplots(indices_df, country_col="country_id", human_rights_col="human_rights_index", environmental_risk_col="enivronmental_risk")

In [ ]:
# from src.visualization import plot_order_volume_by_category
# df = fetch_query_as_dataframe("../sql/article_plot_query.sql")
# plot_order_volume_by_category(df, "Article", "sum_of_order_volume")

In [ ]:
# df = fetch_query_as_dataframe("../sql/industry_plot_query.sql")
# plot_order_volume_by_category(df, "Industry", "sum_of_order_volume")

In [ ]:
# from src.visualization import plot_volume_distribution_by_category
# df = fetch_query_as_dataframe("../sql/supplier_plot_query.sql")
# plot_volume_distribution_by_category(df, category_col="Article", subcategory_col="supplier_id", volume_col="sum_of_order_volume", top_n=5)

# **The Next Steps**
- [How could I improve the MVP?](#improve-the-mvp)
- [What would I change when scaling to 100+ customers?](#scale-to-100-customers)

### **Improve the MVP**

> "We must learn what customers really want, not what we think they should want."  
> ~ Eric Ries

This philosophy emphasizes the importance of listening to customer feedback over relying on assumptions. Without direct feedback from users, any "improvements" risk being misaligned with actual needs. Therefore, I’ve decided not to adjust the Minimum Viable Product (MVP) until feedback becomes available, allowing for changes based on verified user input.

However, if I were building this product for personal use, I’d consider the following enhancements to streamline usability and improve performance:

1. **Minimize Onboarding Friction**
   - Automate column assignment by using language model agent tools such as `langgraph` to simplify the setup process.

2. **Reduce Update Friction**
   - For SAP users:
     - Integrate automated update scheduling using workflow orchestration tools like `Prefect` or `Airflow` to streamline data retrieval from APIs.
   - For Excel users:
     - Eliminate the need to rely on Excel after the initial upload by enabling manual data entry directly into the database through a GUI, or by automating data entry from scanned receipts or other sources.

3. **Improve General Usability**
   - Develop the user interface (UI) with a low-code tool or library, such as `Streamlit` or `Retool`, to enhance ease of use and speed up the development process.

4. **Increase Reliability of Risk Scores**
   - Improve the accuracy of risk scoring by integrating additional external data sources.

5. **Enhance Data Security**
   - Host data securely with a cloud service provider, such as AWS, Azure, or Google Cloud Platform (GCP), to ensure compliance and security.


### **Scale to 100 customers**
- [1. Automating Data Transformation with LLM Agents](#1-automating-data-transformation-with-llm-agents)
- [2. Integrating Automated Update Scheduling](#2-integrating-automated-update-scheduling)
- [3. Multi-Tenant Database Architecture](#3-multi-tenant-database-architecture)
##### **1. Automating Data Transformation with LLM Agents**

One major challenge when scaling to a large number of customers is the continuous manual labor required to transform customer data into a standardized format for analysis. This work typically falls to either:

- The customer, creating a less pleasant user experience.
- The software company, incurring ongoing labor costs.

To streamline this process, I propose:
   - **Predefining a single accepted final data format**: This allows for consistent analysis across customer data.
   - **Automating data conversion** using Large Language Model (LLM) agents: These agents convert diverse data formats provided by customers into the predefined format.
   - **Offering the transformed data to customers** as a suggestion, allowing them to accept or modify it as needed.

![LLM_automate_data_processing](../assets/automate_data_processing.png)

##### **2. Integrating Automated Update Scheduling**

Scheduling and automating data retrieval from APIs can improve efficiency and ensure data freshness. By using workflow orchestration tools like `Prefect` or `Airflow`, this project can manage data retrieval and updates without manual intervention.

![Data Transfer Workflow](../assets/data_transfer.png)

##### **3. Multi-Tenant Database Architecture**

When storing customer information in the cloud, choosing the right database architecture is essential for balancing scalability, security, and performance. The two primary multi-tenant architecture options are:

1. **Single Shared Database with Multiple Schemas**: 
   - For this Minimum Viable Product (MVP), we are using a multi-tenant architecture where:
      - A single database houses both native and customer-specific schemas.
      - The native schema contains tables common to all users.
      - A new schema is created for each customer (currently just one in this MVP).
   - This approach is easier to manage initially but may encounter limitations as the customer base grows.

2. **Dedicated Database per Customer**: 
   - For a more scalable and customer-centric approach, each customer would have a dedicated database, isolating their unique data.
   - This architecture improves data security, performance, and manageability but may increase infrastructure costs and complexity.

For more information on the benefits and drawbacks of these approaches, see the following resources:
   - [Designing Multi-Client Databases](https://www.brentozar.com/archive/2011/06/how-design-multiclient-databases/)
   - [Database Per Customer vs. Single Database for All Customers](https://blog.sqlgrease.com/creating-a-database-per-customer-vs-single-database-for-all-customers-sql-server/)